In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Comment out the below to opt-out of using LangSmith in this notebook. Not required.
# if not os.environ.get("LANGCHAIN_API_KEY"):
#     os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
#     os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Bahrain_2023_Q.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Telemetry LIMIT 10;")

sqlite
['Drivers', 'Event', 'Laps', 'Sessions', 'Telemetry', 'Tracks', 'Weather']


"[(1, 1271, 'VER', 80.0, 7087, 2, 26.0, 0, 8, -89.39, 4453.24, 0.0, 0, '2023-03-04 15:04:00.840000'), (2, 1271, 'VER', 82.0, 7112, 2, 24.0, 0, 8, -87.0, 4498.0, 0.0, 0, '2023-03-04 15:04:01.039000'), (3, 1271, 'VER', 85.0, 7401, 2, 25.0, 0, 8, -85.0, 4564.0, 0.0, 0, '2023-03-04 15:04:01.318000'), (4, 1271, 'VER', 90.0, 7908, 2, 31.0, 0, 8, -83.0, 4608.0, 0.0, 0, '2023-03-04 15:04:01.498000'), (5, 1271, 'VER', 92.0, 8132, 2, 35.0, 0, 8, -81.84, 4636.77, 0.0, 0, '2023-03-04 15:04:01.604000'), (6, 1271, 'VER', 95.0, 8378, 2, 36.0, 0, 8, -80.0, 4681.0, 0.0, 0, '2023-03-04 15:04:01.759000'), (7, 1271, 'VER', 99.0, 8625, 2, 37.0, 0, 8, -78.7, 4704.75, 0.0, 0, '2023-03-04 15:04:01.844000'), (8, 1271, 'VER', 101.0, 8925, 2, 40.0, 0, 8, -77.0, 4736.0, 0.0, 0, '2023-03-04 15:04:01.959000'), (9, 1271, 'VER', 103.0, 9225, 2, 43.0, 0, 8, -75.0, 4793.0, 0.0, 0, '2023-03-04 15:04:02.159000'), (10, 1271, 'VER', 105.0, 9526, 2, 46.0, 0, 8, -74.3, 4810.45, 0.0, 0, '2023-03-04 15:04:02.204000')]"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db, )
response = chain.invoke({"question": "Who is the fastest driver?"})
response

'SQLQuery: \n```sql\nSELECT "driver_name", MIN("lap_time_in_seconds") AS "fastest_lap_time"\nFROM "Laps"\nGROUP BY "driver_name"\nORDER BY "fastest_lap_time"\nLIMIT 1;\n```'

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Only return SQL Query not anything else like ```sql ... ```
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates\
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [11]:
query = full_chain.invoke(
    {
        "question": "WWho is the fastest driver?"
    }
)
print(query)

SELECT "driver_name", MIN("lap_time_in_seconds") AS "fastest_lap_time"
FROM "Laps"
GROUP BY "driver_name"
ORDER BY "fastest_lap_time" ASC
LIMIT 1;


In [12]:
db.run(query)

"[('VER', 89.708)]"

In [13]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [14]:
full_chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [15]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | validation_chain | execute_query
chain.invoke({"question": "Who is the fastest driver?"})

"[('VER', 89.708)]"

## Answer the question


In [17]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | validation_chain | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "Who is the fastest driver?"})

'The fastest driver is VER, with a fastest lap time of 89.708 seconds.'